In [ ]:
#versions: shap==0.48.0, pandas==2.1.1, matplotlib==3.8.0, numpy==1.26.4


In [ ]:
import shap
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
FEATURE_SET = "rBL"#one of "BL", "BL+VOL", "BL+RAD", "BL+VOL+RAD", "rBL", "rBL+VOL", "rBL+RAD", "rBL+VOL+RAD"
DATASET_SEL = "CN+MCI" #one of "CN+MCI", "MCI"
MODEL="CoxnetSurvivalAnalysis" # one of "CoxnetSurvivalAnalysis", "CoxPHSurvivalAnalysis", "ExtraSurvivalTrees", "GradientBoostingSurvivalAnalysis", "RandomSurvivalForest"
SAMPLE_WEIGHTS=True # True or False


In [ ]:
if SAMPLE_WEIGHTS:
    MODEL_DIR = os.path.join("./results/"+MODEL+"sw_"+DATASET_SEL+"_"+FEATURE_SET+"/")
else:
    MODEL_DIR = os.path.join("./results/"+MODEL+"_"+DATASET_SEL+"_"+FEATURE_SET+"/")

In [ ]:
def ipcw_brier_scorer(estimator, X, y):
    scores = []
    for train_idx, test_idx in index_pairs:  # <- diese benutzt du direkt
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        w_tr, _ = weights[train_idx], weights[test_idx]

        estimator.fit(X_tr, y_tr, sample_weight=w_tr)
        surv_fns = estimator.predict_survival_function(X_te)
        preds = np.asarray([[fn(t) for t in [4.0, 8.0]] for fn in surv_fns])

        score = integrated_brier_score(y_tr, y_te, preds, [4.0, 8.0])
        scores.append(score)
    return -np.mean(scores)

In [ ]:
filenameCSV=MODEL_DIR+"/resultsCV.csv"
trainingDSCSV=MODEL_DIR+"/training.csv"
testDSCSV=MODEL_DIR+"/test.csv"
filename=MODEL_DIR+"/model_bayes_optimization.sav"
clf=pickle.load(open(filename, "rb"))
df_ges= pd.read_csv(trainingDSCSV)
df_ges_test= pd.read_csv(testDSCSV)
df_ges=df_ges.set_index(["PTID","IMAGEUID"])
df_ges_test=df_ges_test.set_index(["PTID","IMAGEUID"])
def f(x):
    new=pd.DataFrame(x,columns=df_ges.columns)
    return clf.predict(new)

explainer = shap.Explainer(f, df_ges)
start_time = time.time()
shap_values = explainer(df_ges_test)
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))
print(shap_values.shape[0])
savename_shapley_data=MODEL_DIR+"/ShapValues_test_ADNI.pkl"
with open(savename_shapley_data, "wb") as outp:
    pickle.dump(shap_values, outp)